# Análise de Usuários através de Keystrokes

Nesse trabalho, busco explorar o quanto o modo como um usuário digita diz sobre ele. É possível dicerní-lo de outros apenas por seu padrão de digitação? Conseguimos agrupar usuários que digitam de forma parecida? O que mais é possível extrair de dados de telcas?

## Motivação

Qual a utilidade de ser se estudar padrões de digitação de um usuário?

Assumindo que cada usuário tem um padrão de digitação único, ou pelo menos razoavelmente único, é possível pensar em uma tarefa de identificação. Sites como o Coursera utilizam esses dados justamente para julgar se um aluno realmente está fazendo os deveres do curso. Logo quando um usuário se cadastra, ele deve digitar um parágrafo para que um perfil associado seja criado. Esse perfil é então comparado posteriormente aos textos (códigos ou respostas discursivas) escritos pelo aluno. Por ser utilizado por tanto tempo na plataforma, podemos suspeitar que esse tipo de dado é um bom indicativo de unicidade.

Outra aplicação possível é identificação de usuários fazendo login. Se um sistema for capaz de identificar o usuário pelo modo como digita, uma camada a mais de segurança pode ser criada. Afinal, não basta saber a senha da conta, é necessário digitar como o dono original digita. Dessa forma, fraudes por senhas vazadas, bancos de dados comprometidos, ou até mesmo senhas anotadas em _post-its_ no monitor podem ser evitadas.

Para o caso de uso da senha é interessante notar um detalhe que é o não envio da senha em _plain text_ ao servidor verificador. É extremamente comum - se crucial para uma base mínima de segurança - que a senha de um usuário não trafegue a rede em _plain text_. Isso porque é muito fácil um malfeitor capturar dados transientes em uma rede (principalmente pública). Dessa forma, se a senha não tiver nenhum mecanismo de proteção, a conta do usuário estará comprometida.

O que normalmente ocorre é o envio de um _hash_ da senha. O _hash_ nada mais é do que uma função que leva uma entrada a uma saída de forma simples, mas garante que a inversão (sabendo uma saída, encontrar a entrada correspondente) é uma tarefa árdua. Utilizando esse método, sites que permitem fazer login não precisam guardar as senhas de seus usuários, basta guardar os hashes das senhas. Afinal, como cada senha sempre produz o mesmo hash, é possível comparar hashes para saber se a senha está correta. Aplicando essa metodologia, mesmo que ocorra um vazamento de senhas ou uma senha seja fisgada em trânsito, o segredo está seguro. Apenas o hash foi comprometido e como dito acima, hashes são difíceis de reverter.

O que isso implica na utilização de dados de digitação para verificação do usuário? Imediatamente, significa que não se pode utilizar as teclas nem os tempos de digitação entre elas para tirar tais conclusões. Afinal, se isso fosse utilizado, todo o propósito de não enviar a senha em _plain text_ estará perdido. Soluções mais criativas terão que ser utilizadas, como veremos a frente.

## Dataset

Para começar a exploração desse tópico, escolhi um dataset famoso de dados de _keystrokes_. O dataset é conhecido como _CMU Keystroke Dynamics – Benchmark Data Set_.

### Contrução do dataset

Esse dataset foi montado com dados de 51 pessoas diferentes digitando a senha `.tie5Roanl` repetidas vezes. Ao todo foram 400 repetições por pessoa espaçadas em sessões distintas. Em cada sessão a pessoa digitava a senha 50 vezes, implicando num total de 8 sessões que foram feitas com pelo menos um dia de folga entre elas para capturar a variação de padrões de digitação ao longo de dias.

### Dados coletados

Resta, agora, sabermos que métricas foram efetivamente coletadas e armazenadas nesse dataset. Cada sessão de coleta captura três tipos de dados: `Hold`, `Down-Down`, `Up-Down`.

![Métricas coletadas](data_explanation.jpg)

- Hold: Mede o tempo que uma tecla foi pressionada (desde o momento que foi abaixada até quando é solta).
- KeyDown-KeyDown: Mede o tempo entre uma tecla ser abaixada e a próxima ser abaixada.
- KeyUp-KeyDown: Mede o tempo entre uma tecla ser solta e a próxima abaixada.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
with open("DSL-StrongPasswordData.csv") as f:
    data = np.array([line.split(",") for line in f.read().strip().split("\n")[1:]])

In [11]:
# Formatando os dados
users, keystrokes = data[:,0], data[:,1:].astype(float)
print(users[:2], "\n", keystrokes[:2])

['s002' 's002'] 
 [[ 1.      1.      0.1491  0.3979  0.2488  0.1069  0.1674  0.0605  0.1169
   0.2212  0.1043  0.1417  1.1885  1.0468  0.1146  1.6055  1.4909  0.1067
   0.759   0.6523  0.1016  0.2136  0.112   0.1349  0.1484  0.0135  0.0932
   0.3515  0.2583  0.1338  0.3509  0.2171  0.0742]
 [ 1.      2.      0.1111  0.3451  0.234   0.0694  0.1283  0.0589  0.0908
   0.1357  0.0449  0.0829  1.197   1.1141  0.0689  0.7822  0.7133  0.157
   0.7877  0.6307  0.1066  0.1684  0.0618  0.1412  0.2558  0.1146  0.1146
   0.2642  0.1496  0.0839  0.2756  0.1917  0.0747]]
